In [9]:
%pip install tqdm 

Note: you may need to restart the kernel to use updated packages.


In [20]:
import pandas as pd
from pypfopt import EfficientFrontier, risk_models, expected_returns
import yfinance as yf
import itertools
import json
from tqdm import tqdm
import numpy as np

In [11]:
def portfolio_settings(risk_level: int, years_to_target: int) -> float:
    """
    Estimate a target volatility based on risk level (1-5) and years to target date.

    Args:
        risk_level (int): Risk tolerance (1 = conservative, 5 = aggressive)
        years_to_target (int): Time horizon in years

    Returns:
        float: Target annualized volatility (0.0 - 1.0)
    """

    if not (1 <= risk_level <= 5):
        raise ValueError("risk_level must be between 1 and 5")

    if years_to_target <= 0:
        raise ValueError("years_to_target must be positive")
    
    base_vol_map = {
        1: 0.06,  # conservative
        2: 0.10,
        3: 0.14,
        4: 0.18,
        5: 0.22,  # aggressive
    }
    base_vol = base_vol_map[risk_level]

    # Volatility decays nonlinearly as you near the target date
    glide_factor = max((years_to_target / 40) ** 0.5, 0.2)
    adjusted_vol = base_vol * glide_factor

    adjusted_vol = max(adjusted_vol, .04)
    return round(adjusted_vol, 4)


In [12]:
# Define the list of ticker symbols
tickers = ['VTI', 'BND', 'VXUS', 'BNDX', 'BITB']  # Add or remove tickers as needed

# Download historical data for the specified tickers
data = yf.download(tickers, start='1995-01-01', end='2025-04-22', interval='1d', auto_adjust=True)

# Extract the 'Close' prices
close_prices = data['Close']

# Reset the index to have 'Date' as a column
close_prices.reset_index(inplace=True)

# Save the data to a CSV file
close_prices.to_csv('sample_data.csv', index=False)


[*********************100%***********************]  5 of 5 completed


In [13]:
# Load adjusted close prices
csv_path = "sample_data.csv"
prices = pd.read_csv(csv_path, index_col=0, parse_dates=True)

# Ensure tickers match column order
tickers = list(prices.columns)


# Step 1: Calculate expected returns and sample covariance matrix
mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)

# Step 2: Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S, weight_bounds=(0, 0.4))

if "BITB" in tickers:
    ef.add_constraint(lambda w: w[tickers.index("BITB")] <= 0.15)

if "SPMO" in tickers:
    ef.add_constraint(lambda w: w[tickers.index("SPMO")] <= 0.25)

if "AVUV" in tickers:
    ef.add_constraint(lambda w: w[tickers.index("AVUV")] <= 0.25)

if "FVAL" in tickers:
    ef.add_constraint(lambda w: w[tickers.index("FVAL")] <= 0.3)
weights = ef.efficient_risk(vol)



cleaned_weights = ef.clean_weights()

# Step 3: Output weights and performance
print("Optimized Portfolio Weights:")
print(cleaned_weights)

performance = ef.portfolio_performance(verbose=True)


NameError: name 'vol' is not defined

In [ ]:
def buildPortfolio(riskLevel:int, yearsToTarget:int, bitcoin_focus: bool, value_focus: bool, momentum_focus: bool, small_cap_focus: bool):
    tickers = ["VTI", "BND", "BNDX", "VXUS", "VWO", "VEA", "USO", "VNQ", "QQQM"]

    if bitcoin_focus:
        tickers.append("BITB")
    if value_focus:
        tickers.append("VTV")
    if momentum_focus:
        tickers.append("SPMO")
    if small_cap_focus:
        tickers.append("AVUV")

    prices = yf.download(
        tickers, 
        start='1995-01-01', 
        end='2025-04-22', 
        interval='1d', 
        auto_adjust=True
    )['Close']

    # Ensure Date is a datetime index (it should already be)
    prices.index = pd.to_datetime(prices.index)

    # Get list of tickers (column names)
    tickers = list(prices.columns)

    mu = expected_returns.mean_historical_return(prices)
    S = risk_models.sample_cov(prices)

    ef = EfficientFrontier(mu, S, weight_bounds=(0, 0.7))

    ef.add_constraint(lambda w: w[tickers.index("QQQM")] <= 0.22)

    if "BITB" in tickers:
        ef.add_constraint(lambda w: w[tickers.index("BITB")] <= 0.15)

    if "SPMO" in tickers:
        ef.add_constraint(lambda w: w[tickers.index("SPMO")] <= 0.15)

    if "AVUV" in tickers:
        ef.add_constraint(lambda w: w[tickers.index("AVUV")] <= 0.25)

    if "VTV" in tickers:
        ef.add_constraint(lambda w: w[tickers.index("VTV")] <= 0.3)
    
    vol = portfolio_settings(risk_level=riskLevel, years_to_target=yearsToTarget)
    
    ef.efficient_risk(vol)

    cleaned_weights = ef.clean_weights()

    # Step 3: Output weights and performance
    # print("Optimized Portfolio Weights:")
    # print(cleaned_weights)

    # performance = ef.portfolio_performance(verbose=True)
    # print(performance)
    return cleaned_weights
    

In [ ]:
portfolio = buildPortfolio(5,30,True, True, True, True)

portfolio

In [ ]:
# Define ranges
a_range = range(1, 6)              # 1 to 5
b_range = range(1, 44)             # 1 to 43
bool_values = [True, False]        # For focus values

# Generate all combinations
combinations = itertools.product(
    a_range,
    b_range,
    bool_values,
    bool_values,
    bool_values,
    bool_values
)

# Collect results
results = []

# buildPortfolio(1,1,True,True,True,True)
for combo in tqdm(combinations, total=5 * 43 * 2**4, desc="Building Portfolios"):
# for combo in itertools.islice(combinations, 12):
    # combo = combinations[i]
    risk, yte, btc, value, momentum, smallcap = combo
    # print(combo)
    weights = buildPortfolio(*combo)
    holdings = [{"ticker": k, "percentage": round(v * 100, 2)} for k, v in weights.items() if v > 0.0]

    portfolio_json = {
        "years_to_expiration": yte,
        "risk_aptitude": risk,
        "bitcoin_focus": btc,
        "smallcap_focus": smallcap,
        "value_focus": value,
        "momentum_focus": momentum,
        "holdings": holdings
    }
    results.append(portfolio_json)

# Save to JSON file
with open("portfolios.json", "w") as f:
    json.dump(results, f, indent=2)

print(f"Saved {len(results)} portfolios to 'portfolios.json'")

In [22]:
def omega_ratio(tickers, weights, start_date="2020-01-01", end_date=None, r_thresh=0.0):
    """
    Calculate the Omega Ratio for a portfolio of tickers with given weights.
    
    Parameters:
        tickers (list of str): Ticker symbols.
        weights (list of float): Corresponding weights (should sum to 1).
        start_date (str): Start date for historical data (YYYY-MM-DD).
        end_date (str or None): End date for historical data (default today).
        r_thresh (float): Threshold return (e.g. 0 or risk-free rate).
        
    Returns:
        float: Omega Ratio of the portfolio.
    """
    if len(tickers) != len(weights):
        raise ValueError("Tickers and weights must have the same length.")
    if not np.isclose(sum(weights), 1.0):
        raise ValueError("Weights must sum to 1.")
    
    # Fetch adjusted close prices
    data = yf.download(tickers, start=start_date, end=end_date)["Close"]
    data = data.dropna()
    
    # Compute daily log returns
    log_returns = np.log(data / data.shift(1)).dropna()
    
    # Compute portfolio returns
    portfolio_returns = log_returns.dot(weights)
    
    # Split returns into gains and losses based on threshold
    gains = portfolio_returns[portfolio_returns > r_thresh] - r_thresh
    losses = r_thresh - portfolio_returns[portfolio_returns < r_thresh]
    
    # Avoid division by zero
    if losses.sum() == 0:
        return np.inf
    
    omega = gains.sum() / losses.sum()
    return omega

In [ ]:
tickers = ['AAPL', 'MSFT', 'GOOGL']
weights = [0.4, 0.3, 0.3]
ratio = omega_ratio(tickers, weights, start_date="2022-01-01")
print("Omega Ratio:", ratio)

# Our portfolio

In [24]:
portfolio = buildPortfolio(4,30,False, False, False, False)
tickers = []
weights = []
for key in portfolio:
    if portfolio[key] > 0:
        tickers.append(key)
        weights.append(portfolio[key])
ratio = omega_ratio(tickers, weights, start_date="2022-01-01")
print("Omega Ratio:", ratio)

[*********************100%***********************]  9 of 9 completed
/Users/clinthart/Desktop/code/Evergreen/model/.venv/lib/python3.13/site-packages/pypfopt/risk_models.py:70: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(
[*********************100%***********************]  3 of 3 completed

Omega Ratio: 1.041093125100601


# Vanguard Portfolio

In [25]:
tickers = ["VTI", "VXUS", "BND", "BNDX"]
weights = [0.53, 0.37, 0.07, 0.03]
ratio = omega_ratio(tickers, weights, start_date="2022-01-01")
print("Omega Ratio:", ratio)

[*********************100%***********************]  4 of 4 completed

Omega Ratio: 0.99791923691425
